In [ ]:
# third party
import pytest
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy
from syft.abstract_node import NodeType
from syft.service.network.routes import HTTPNodeRoute
from syft.service.response import SyftAttributeError
from syft.service.response import SyftSuccess

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_PORT = 9083

In [ ]:
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
canada_enclave = sy.orchestra.launch(
    name="canada-enclave",
    node_type=NodeType.ENCLAVE,
    port=CANADA_ENCLAVE_PORT,
    dev_mode=True,
    reset=True,
)

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.node_type == NodeType.DOMAIN
assert do_italy_client.metadata.node_type == NodeType.DOMAIN

In [ ]:
# Using public datasets from Freely Extensible Biomedical Record Linkage (Febrl) project
canada_census_data, italy_census_data = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data, italy_census_data],
    [do_canada_client, do_italy_client],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

In [ ]:
assert len(do_canada_client.datasets.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

In [ ]:
# Create accounts for Data Scientists
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

In [ ]:
# Register enclaves
route = HTTPNodeRoute(host_or_ip="localhost", port=CANADA_ENCLAVE_PORT)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

In [ ]:
ds_canada_client = sy.login(
    email="sheldon@caltech.edu", password="changethis", port=CANADA_DOMAIN_PORT
)

In [ ]:
with pytest.raises(SyftAttributeError):
    # Data scientist is not allowed to add enclave to the domain
    ds_canada_client.enclaves.add(
        name="Dummy Enclave", route=HTTPNodeRoute(host_or_ip="localhost", port=9084)
    )

In [ ]:
assert (len(ds_canada_client.enclaves.get_all())) == 1
ds_canada_client.enclaves.get_all()

In [ ]:
# Cleanup local domain servers
if canada_node.node_type.value == "python":
    canada_node.land()

if italy_node.node_type.value == "python":
    italy_node.land()

if canada_enclave.node_type.value == "python":
    canada_enclave.land()